In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
import statsmodels.api as sm
import re
import plotly.express as px
DATA_DIR = Path("..") / "Datasets" / "Cleaned Data"

In [47]:
cd_2020 = pd.read_csv(DATA_DIR / "cleaned_2020.csv", encoding="latin1")

#### 1. Respondent Demographics


In [11]:
## 1a) Age and gender Distribution

SURVEY_YEAR = 2020

viz_demo_df = (
    cd_2020[["birth_year", "gender"]]
    .dropna()
    .assign(age=lambda d: SURVEY_YEAR - d["birth_year"])
)

age_bins = [18, 25, 35, 45, 55, 65, 100]
age_labels = ["18–24", "25–34", "35–44", "45–54", "55–64", "65+"]

viz_demo_df["age_group"] = pd.cut(
    viz_demo_df["age"],
    bins=age_bins,
    labels=age_labels,
    right=False
)

age_gender_dist = (
    viz_demo_df
    .groupby(["age_group", "gender"])
    .size()
    .reset_index(name="count")
)

age_dist = px.bar(
    age_gender_dist,
    x="age_group",
    y="count",
    color="gender",
    barmode="stack",
    title="Age Distribution by Gender (Stacked)",
    labels={
        "age_group": "Age Group",
        "count": "Number of Respondents",
        "gender": "Gender"
    },
    color_discrete_map={
        "Male": "skyblue",
        "Female": "pink",
        "Rather not say": "dark green"
    }
)

age_dist.show()

C:\Users\Natasha\AppData\Local\Temp\ipykernel_12648\964590735.py:23: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [13]:
## 1b) Industry representation
viz_industry_df = (
    cd_2020[["industry", "industry_detailed"]]
    .dropna()
)

industry_tree = (
    viz_industry_df
    .groupby(["industry", "industry_detailed"])
    .size()
    .reset_index(name="count")
)

ind_dist = px.treemap(
    industry_tree,
    path=["industry", "industry_detailed"],
    values="count",
    title="Industry Distribution (Detailed Breakdown)"
)

industry_tree["percent"] = (
    industry_tree["count"] / industry_tree["count"].sum() * 100
)

ind_dist = px.treemap(
    industry_tree,
    path=["industry", "industry_detailed"],
    values="count",
    custom_data=["percent"],
    title="Industry Distribution (Detailed Breakdown)"
)

ind_dist.update_traces(
    hovertemplate=
        "<b>%{label}</b><br>" +
        "Respondents: %{value}<br>" +
        "Share of sample: %{customdata[0]:.1f}%<extra></extra>"
)

ind_dist.show()



In [15]:
## 1c) Company size and region
location_colors = {
    "Metro": "#1f77b4",  # strong blue
    "Region": "#6baed6",  # lighter blue
    # Add more locations here
}

viz_org_loc_df = (
    cd_2020[["org_size", "location"]]
    .dropna()
)

org_loc_dist = (
    viz_org_loc_df
    .groupby(["org_size", "location"])
    .size()
    .reset_index(name="count")
)

fig3 = px.bar(
    org_loc_dist,
    x="org_size",
    y="count",
    color="location",
    barmode="stack",
    color_discrete_map=location_colors,
    title="Company Size Distribution by Location",
    labels={
        "org_size": "Company Size",
        "count": "Respondents",
        "location": "Location"
    }
)
fig3.update_xaxes(
    tickangle=0,      # straight, not slanted
    tickmode="array", # ensures categorical labels
    tickvals=org_loc_dist["org_size"].unique(),
    tickfont=dict(
        size=12,
        family="Arial"
    )
)
fig3.update_traces(
    hovertemplate="<b>%{x}</b><br>%{color}: %{y}<extra></extra>"
)


fig3.show()

##### ii) Key relationships in Demographics against remote work

In [ ]:
SURVEY_YEAR = 2020

# Select relevant columns and drop missing
viz_demo_df = cd_2020[
    ["birth_year", "gender", "org_size", "industry",
     "remote_work_pct_last_year", "remote_work_pct_last_3_months", "remote_work_pref_pct_future"]
].dropna()

# Compute age and age groups
viz_demo_df["age"] = SURVEY_YEAR - viz_demo_df["birth_year"]
age_bins = [18, 25, 35, 45, 55, 65, 100]
age_labels = ["18–24", "25–34", "35–44", "45–54", "55–64", "65+"]
viz_demo_df["age_group"] = pd.cut(
    viz_demo_df["age"], bins=age_bins, labels=age_labels, right=False
)

viz_long = viz_demo_df.melt(
    id_vars=["age_group", "gender", "org_size", "industry"],
    value_vars=[
        "remote_work_pct_last_year",
        "remote_work_pct_last_3_months",
        "remote_work_pref_pct_future"
    ],
    var_name="period",
    value_name="remote_pct"
)

# Rename periods for readability
period_map = {
    "remote_work_pct_last_year": "Pre-COVID",
    "remote_work_pct_last_3_months": "During-COVID",
    "remote_work_pref_pct_future": "Future Preference"
}
viz_long["period"] = viz_long["period"].map(period_map)

# Age group x Remote work
fig_age = px.box(
    viz_long,
    x="age_group",
    y="remote_pct",
    color="period",
    points="all",  # shows individual points
    title="Remote Work % by Age Group Across Periods",
    labels={"age_group": "Age Group", "remote_pct": "Remote Work %", "period": "Period"},
    color_discrete_sequence=px.colors.qualitative.Pastel
)
fig_age.show()


In [ ]:
# Gender x Remote Work
fig_gender = px.box(
    viz_long,
    x="gender",
    y="remote_pct",
    color="period",
    points="all",
    title="Remote Work % by Gender Across Periods",
    labels={"gender": "Gender", "remote_pct": "Remote Work %", "period": "Period"},
    color_discrete_sequence=px.colors.qualitative.Pastel
)
fig_gender.show()


In [20]:
# Company size x Remote Work
fig_size = px.box(
    viz_long,
    x="org_size",
    y="remote_pct",
    color="period",
    points="all",
    title="Remote Work % by Company Size Across Periods",
    labels={"org_size": "Company Size", "remote_pct": "Remote Work %", "period": "Period"},
    color_discrete_sequence=px.colors.qualitative.Pastel
)
# Optional: reorder sizes if textual
size_order = ["1–10", "11–50", "51–200", "201–500", "500+"]
fig_size.update_xaxes(categoryorder="array", categoryarray=size_order)
fig_size.show()

In [26]:
# 1. Compute top & bottom 2 industries
industry_summary = (
    viz_long
    .groupby("industry", as_index=False)["remote_pct"]
    .mean()
    .sort_values("remote_pct")
)

selected_industries = pd.concat([
    industry_summary.head(2)["industry"],
    industry_summary.tail(2)["industry"]
])

# 2. Filter dataset
viz_industry_filtered = viz_long[
    viz_long["industry"].isin(selected_industries)
]

# 3. Plot
import plotly.express as px

fig_industry = px.box(
    viz_industry_filtered,
    x="industry",
    y="remote_pct",
    color="period",
    points="all",
    title="Remote Work % by Selected Industries (Top 2 vs Bottom 2)",
    labels={
        "industry": "Industry",
        "remote_pct": "Remote Work %",
        "period": "Period"
    },
    color_discrete_sequence=px.colors.qualitative.Pastel
)

fig_industry.show()


#### HYPOTHESIS TESTING

##### A) Reality vs Preference
-- (Pre-COVID alignment)

Hypothesis 1a: There is a significant difference between actual remote work last year (pre-COVID) and the remote work respondents would have preferred for that same period.

-- (COVID period alignment)

Hypothesis: There is a significant difference between actual remote work in the last 3 months and the remote work respondents would have preferred during the same period.

-- (Future vs recent COVID behavior):

Hypothesis 1c: There is a significant difference between actual remote work in the last 3 months and respondents’ preferred remote work in the future.

Conclusion:

a) Pre-COVID, actual remote work was less than respondents wanted, indicating some unmet demand.

b) During the last 3 months, actual remote work was about right, matching what respondents were comfortable with.

c) For the future, respondents expect slightly less remote work than they experienced during COVID, suggesting a desire to rebalance towards hybrid arrangements rather than full-time remote work.

###### The data is paired, bounded and survey-based thus Wilcoxon signed-rank is the right statistical test to use


In [34]:
analysis_df = cd_2020[
    [
        "remote_work_pct_last_year",
        "remote_work_pref_pct_last_year",
        "remote_work_pct_last_3_months",
        "remote_work_pref_pct_last_3_months",
        "remote_work_pref_pct_future"
    ]
].dropna()

analysis_df["gap_precovid"] = (
    analysis_df["remote_work_pref_pct_last_year"]
    - analysis_df["remote_work_pct_last_year"]
)

analysis_df["gap_covid"] = (
    analysis_df["remote_work_pref_pct_last_3_months"]
    - analysis_df["remote_work_pct_last_3_months"]
)

analysis_df["gap_future_vs_recent"] = (
    analysis_df["remote_work_pref_pct_future"]
    - analysis_df["remote_work_pct_last_3_months"]
)


In [ ]:
wilcox_precovid = wilcoxon(analysis_df["gap_precovid"])
wilcox_covid = wilcoxon(analysis_df["gap_covid"])
wilcox_future = wilcoxon(analysis_df["gap_future_vs_recent"])

wilcox_precovid, wilcox_covid, wilcox_future


In [65]:
gap_df = analysis_df.melt(
    value_vars=["gap_precovid", "gap_covid", "gap_future_vs_recent"],
    var_name="period_gap",
    value_name="gap"
)

fig = px.box(
    gap_df,
    x="period_gap",
    y="gap",
    points=False,
    title="Remote Work Preference Gaps Across Periods",
    labels={
        "period_gap": "Gap Type",
        "gap": "Preferred − Actual Remote Work (%)"
    }
)
fig.add_hline(y=0, line_dash="dash", line_color="red")
fig.show()


##### WHY THE BIG GAP?

#### B. Organizational support & preparedness

Hypothesis 2: H1: Employee perceptions of organizational support for remote work differ across time periods (Last Year, Last 3 Months, Future)

**Conclusion**:
Employee perceptions of organizational support for remote work have changed selectively over time. Encouragement and ease of obtaining permission improved in the most recent period, while preparedness, common practice, collaboration, and recommendation remained stable. These findings suggest that organizations have become more supportive in certain aspects of remote work, but other practices have not changed.

In [ ]:
# Columns
org_cols = [
    # Last Year
    "remote_last_year_org_encouraged_agreement",
    "remote_last_year_org_prepared_agreement",
    "remote_last_year_common_practice_agreement",
    "remote_last_year_permission_easy_agreement",
    "remote_last_year_collaboration_easy_agreement",
    "remote_last_year_recommend_agreement",

    # Last 3 Months
    "remote_last_3_months_org_encouraged_agreement",
    "remote_last_3_months_org_prepared_agreement",
    "remote_last_3_months_common_practice_agreement",
    "remote_last_3_months_permission_easy_agreement",
    "remote_last_3_months_collaboration_easy_agreement",
    "remote_last_3_months_recommend_agreement",

    # Future (use exact original names)
    "Imagine that COVID-19 is cured or eradicated. How likely would you consider the following statements? - My employer would encourage more remote working",
    "Imagine that COVID-19 is cured or eradicated. How likely would you consider the following statements? - My employer would make changes to support remote working",
    "Imagine that COVID-19 is cured or eradicated. How likely would you consider the following statements? - I would have more choice about whether I work remotely"
]

# Melt to long format
df_long = cd_2020[org_cols].melt(
    var_name="question_period",
    value_name="score"
)

# Extract period
def get_period(col_name):
    if "last_year" in col_name:
        return "Last Year"
    elif "last_3_months" in col_name:
        return "Last 3 Months"
    else:
        return "Future"

df_long["period"] = df_long["question_period"].apply(get_period)

# Simplify question names (remove the period/time part)
def clean_question(col_name):
    if "last_year" in col_name or "last_3_months" in col_name:
        return "_".join(col_name.split("_")[3:])
    else:
        # Future: just take the main text after the dash
        return col_name.split("-")[-1].strip()

df_long["question"] = df_long["question_period"].apply(clean_question)

# Plot boxplots
fig = px.box(
    df_long,
    x="question",
    y="score",
    color="period",
    points="all",
    title="Organizational Support & Remote Work Experience Across Periods",
    labels={
        "question": "Organizational Support Question",
        "score": "Score (1=Low, 5=High)",
        "period": "Period"
    },
    color_discrete_sequence=px.colors.qualitative.Pastel
)

fig.update_xaxes(tickangle=-45)
fig.show()


In [60]:
import numpy as np
import pandas as pd
from scipy.stats import friedmanchisquare, wilcoxon

results = []

for question in df_long["question"].unique():
    # Get all columns corresponding to this question
    cols = [col for col in org_cols if question in col or question in col.split("-")[-1].strip()]
    data = cd_2020[cols].dropna()
    
    n_cols = len(data.columns)
    
    if n_cols >= 3:
        # Friedman for 3+ periods
        stat, p = friedmanchisquare(*[data[col] for col in data.columns])
        test_type = "Friedman"
    elif n_cols == 2:
        # Wilcoxon for 2 periods
        stat, p = wilcoxon(data[data.columns[0]], data[data.columns[1]])
        test_type = "Wilcoxon"
    else:
        continue
    
    # Compute medians for available periods
    medians = data.median().to_dict()
    
    # Compute direction arrows for available periods
    cols_sorted = list(data.columns)
    directions = []
    for i in range(len(cols_sorted)-1):
        a, b = medians[cols_sorted[i]], medians[cols_sorted[i+1]]
        if b > a:
            directions.append("↑")
        elif b < a:
            directions.append("↓")
        else:
            directions.append("=")
    direction_str = " → ".join(directions)
    
    results.append({
        "question": question,
        "test": test_type,
        "p_value": p,
        "n_periods": n_cols,
        "direction": direction_str,
        **medians
    })

summary_df = pd.DataFrame(results)
summary_df


,question,test,p_value,n_periods,direction,remote_last_year_org_encouraged_agreement,remote_last_3_months_org_encouraged_agreement,remote_last_year_org_prepared_agreement,remote_last_3_months_org_prepared_agreement,remote_last_year_common_practice_agreement,remote_last_3_months_common_practice_agreement,remote_last_year_permission_easy_agreement,remote_last_3_months_permission_easy_agreement,remote_last_year_collaboration_easy_agreement,remote_last_3_months_collaboration_easy_agreement,remote_last_year_recommend_agreement,remote_last_3_months_recommend_agreement
0,org_encouraged_agreement,Wilcoxon,6.809617e-79,2,↑,3.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,org_prepared_agreement,Wilcoxon,1.240737e-24,2,=,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,common_practice_agreement,Wilcoxon,8.302864e-75,2,=,NaN,NaN,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
3,permission_easy_agreement,Wilcoxon,4.892808e-53,2,↑,NaN,NaN,NaN,NaN,NaN,NaN,4.0,5.0,NaN,NaN,NaN,NaN
4,collaboration_easy_agreement,Wilcoxon,4.981944e-21,2,=,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,NaN
5,recommend_agreement,Wilcoxon,4.237594e-18,2,=,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0


##### WHY COMPANIES SHOULD CARE

#### C. PRODUCTIVITY AND WORK-LIFE BALANCE

Hypothesis 3: Employees who spend more time working remotely allocate their working hours more efficiently, resulting in higher effective work time and lower commuting/preparation time, compared to onsite work.

**Conclusion**: Employees working remotely spend significantly less time commuting and more time on productive work than when onsite, while also having increased time for personal and caring responsibilities. This confirms the hypothesis that remote work improves effective work-hour allocation, which is a direct business advantage.

###### By supporting remote work, companies can increase productivity, reduce employee commuting stress, and improve work-life balance, aligning with employee preferences and the observed gap in remote work.

In [62]:
# Select necessary columns
onsite_cols = ["onsite_commute_hours", "onsite_work_hours", "onsite_personal_hours", "onsite_caring_hours"]
remote_cols = ["remote_commute_hours", "remote_work_hours", "remote_personal_hours", "remote_caring_hours"]

# Melt into long format for plotting
df_time = pd.concat([
    cd_2020[onsite_cols].assign(work_type="Onsite"),
    cd_2020[remote_cols].assign(work_type="Remote")
], axis=0, ignore_index=True)

# Standardize column names
df_time.rename(columns={
    "onsite_commute_hours":"Commute",
    "onsite_work_hours":"Work",
    "onsite_personal_hours":"Personal",
    "onsite_caring_hours":"Caring",
    "remote_commute_hours":"Commute",
    "remote_work_hours":"Work",
    "remote_personal_hours":"Personal",
    "remote_caring_hours":"Caring"
}, inplace=True)

# Melt long for plotting
df_time_long = df_time.melt(id_vars="work_type", value_vars=["Commute","Work","Personal","Caring"],
                            var_name="Activity", value_name="Hours")

# Compute Wilcoxon for paired activities
wilcoxon_results = {}
for activity in ["Commute","Work","Personal","Caring"]:
    stat, p = wilcoxon(cd_2020["onsite_"+activity.lower()+"_hours"], cd_2020["remote_"+activity.lower()+"_hours"])
    wilcoxon_results[activity] = {"statistic": stat, "p_value": p}

# Convert to DataFrame
wilcoxon_df = pd.DataFrame(wilcoxon_results).T.reset_index().rename(columns={"index":"Activity"})
wilcoxon_df


,Activity,statistic,p_value
0,Commute,50753.5,1.940891e-160
1,Work,101422.5,1.827500e-23
2,Personal,95328.0,9.440718e-64
3,Caring,64040.5,3.352898e-26


In [67]:
# Compute average hours per activity per work type
avg_df = df_time_long.groupby(["work_type","Activity"], as_index=False)["Hours"].mean()

# Create a clean bar chart of averages
fig = px.bar(
    avg_df,
    x="Activity",
    y="Hours",
    color="work_type",
    barmode="group",
    text="Hours",
    title="Average Hours Spent per Activity: Remote vs Onsite",
    color_discrete_sequence=px.colors.qualitative.Pastel
)

fig.update_traces(texttemplate='%{text:.1f}', textposition='outside')
fig.show()